<a href="https://colab.research.google.com/github/ddoddii/Mycolab/blob/master/HW4_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


2. AR

- 적절한 형태로 데이터 구성

- 지지도, 신뢰도, Lift 관점에서 규칙을 추출하여 설명

- 결과 해석

# Association Rule(연관분석)

마트에서 소비자들이 어떤 구매 패턴으로 물건을 구매하는지는 마케터나 데이터 분석가들에게 매우 중요한 문제이다. 어떤상품을 어떻게 진열할 것인지 어떤 시기에 어떤 고객을 타겟으로 마케팅을 펼칠지에 따라서 매출이 달라질 수 있다. 따라서 데이터 마이닝에서 배운 연관분석을 바탕으로 장보기 데이터를 활용하여 연관분석에 대해 알아보고 어떠한 연관성이 있는지 분석해보고자 한다. 

데이터는 다음 사이트를 참고했다.

https://www.kaggle.com/code/atasaygin/groceries-market-basket-and-rfm-analysis


Member_number: 구매를 한 사람의 고유 ID 번호

Date: 구매를 한 날짜

itemDescription: 구매한 품목

연관규칙 분석을 통해 어떠한 품목을 같이 구매하는지를 분석하고자 하였다.




## Step1. 데이터 불러오기


연관분석을 수행하기 위해 mixtend 라이브러리를 활용하였다. 
mlxtend: Machine learning extensions. Machine learning에 사용하는 알고리즘들을 제공하는 파이썬 라이브러리로, 연관규칙 분석 시 사용하는 함수도 포함. 


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth
import os
import plotly.express as px

In [ ]:
!pip install fpgrowth

  Created wheel for fpgrowth: filename=fpGrowth-1.0.0-py3-none-any.whl size=2866 sha256=8fe5ee1abeb66dd0b713f8addc6d187c0c8064366d9c067ef74296a9f3eb5385
  Stored in directory: /root/.cache/pip/wheels/64/33/72/1991a9117d1813325c4ef85597ba8ece8c4780adc240bd0b0f
Successfully built fpgrowth


In [ ]:
df=pd.read_csv('/content/drive/Shareddrives/데마 팀플 2/HW4/Groceries_dataset.csv')   #데이터셋 불러오기
print(df)

       Member_number        Date        itemDescription
0               1808  21-07-2015         tropical fruit
1               2552  05-01-2015             whole milk
2               2300  19-09-2015              pip fruit
3               1187  12-12-2015       other vegetables
4               3037  01-02-2015             whole milk
...              ...         ...                    ...
38760           4471  08-10-2014          sliced cheese
38761           2022  23-02-2014                  candy
38762           1097  16-04-2014               cake bar
38763           1510  03-12-2014  fruit/vegetable juice
38764           1521  26-12-2014               cat food

[38765 rows x 3 columns]


In [ ]:
df.head(10)

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
5,4941,14-02-2015,rolls/buns
6,4501,08-05-2015,other vegetables
7,3803,23-12-2015,pot plants
8,2762,20-03-2015,whole milk
9,4119,12-02-2015,tropical fruit


In [ ]:
df.Date =pd.to_datetime(df.Date)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Member_number    38765 non-null  int64         
 1   Date             38765 non-null  datetime64[ns]
 2   itemDescription  38765 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 908.7+ KB


## Step2. 데이터 시각화

In [ ]:

Sales_weekly = df.resample('w', on='Date').size()
fig = px.line(df, x=Sales_weekly.index, y=Sales_weekly,
              labels={'y': 'Number of Sales',
                     'x': 'Date'})
fig.update_layout(title_text='Number of Sales Weekly',
                  title_x=0.5, title_font=dict(size=18)) 
fig.show()

* 주별 판매 물건 수

300~450을 사이로 주별로 다른 판매를 보여준다.

In [ ]:
Unique_customer_weekly = df.resample('w', on='Date').Member_number.nunique()
fig = px.line(Unique_customer_weekly, x=Unique_customer_weekly.index, y=Unique_customer_weekly,
              labels={'y': 'Number of Customers'})
fig.update_layout(title_text='Number of Customers Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.show()

* 주별 손님 수

120~180 사이의 손님이 주별로 방문한다, 

In [ ]:
Sales_per_Customer = Sales_weekly / Unique_customer_weekly
fig = px.line(Sales_per_Customer, x=Sales_per_Customer.index, y=Sales_per_Customer,
              labels={'y': 'Sales per Customer Ratio'})
fig.update_layout(title_text='Sales per Customer Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.update_yaxes(rangemode="tozero")
fig.show()

* 손님 당 구매 물건 갯수 : 손님 당 평균적으로 3개 물건을 구매한 것을 알 수 있음
* 2015년 이후 구매 물건 개수가 증가한 것을 확인할 수 있다.

In [ ]:
Frequency_of_items = df.groupby(pd.Grouper(key='itemDescription')).size().reset_index(name='count')
fig = px.treemap(Frequency_of_items, path=['itemDescription'], values='count')
fig.update_layout(title_text='Frequency of the Items Sold',
                  title_x=0.5, title_font=dict(size=18)
                  )
fig.update_traces(textinfo="label+value")
fig.show()

* 가장 판매가 많이 된 물건 : whole milk, other vegetables, rolls/buns 순서

In [ ]:
user_item = df.groupby(pd.Grouper(key='Member_number')).size().reset_index(name='count') \
    .sort_values(by='count', ascending=False)
fig = px.bar(user_item.head(25), x='Member_number', y='count',
             labels={'y': 'Number of Sales',
                     'count': 'Number of Items Bought'},
             color='count')
fig.update_layout(title_text='Top 20 Customers regarding Number of Items Bought',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='Blue', width=10)))
fig.show()

In [ ]:
pd.DataFrame(df['Member_number'].value_counts().sort_values(ascending=False))[:10]

,Member_number
3180,36
3050,33
2051,33
3737,33
2625,31
3915,31
2433,31
2271,31
3872,30
2394,29


* 3180 손님이 구매를 가장 많이 하였고 (36번) 그 뒤로 3050 (33번), 2051 (33), 3737 (33번) 손님 순으로 구매하였다. 

## Step 3. AR 적용을 위한 데이터 전처리

- `mlxtend`에서 제공하는 [`TransactionEncoder`](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/)을 사용하여 list 형식으로 구성된 거래 데이터를 2차원 numpy array로 변경하였다. 

In [ ]:
df_tmp = df.groupby('Member_number')['itemDescription'].apply(set).apply(list).to_list()
print(df_tmp[:10])

[['yogurt', 'misc. beverages', 'sausage', 'pickled vegetables', 'salty snack', 'pastry', 'whole milk', 'soda', 'hygiene articles', 'canned beer', 'semi-finished bread'], ['white bread', 'whipped/sour cream', 'sausage', 'rolls/buns', 'curd', 'frankfurter', 'whole milk', 'soda', 'beef'], ['other vegetables', 'sugar', 'butter', 'whole milk', 'tropical fruit', 'butter milk', 'frozen vegetables', 'specialty chocolate'], ['sausage', 'rolls/buns', 'root vegetables', 'dental care', 'frozen meals', 'detergent'], ['other vegetables', 'frozen fish', 'chocolate', 'shopping bags', 'rolls/buns', 'dish cleaner', 'red/blush wine', 'root vegetables', 'pastry', 'whole milk', 'packaged fruit/vegetables', 'pip fruit', 'tropical fruit', 'hygiene articles', 'canned beer', 'cling film/bags'], ['rolls/buns', 'whipped/sour cream', 'margarine'], ['chocolate', 'shopping bags', 'rolls/buns', 'bottled water', 'frankfurter', 'flour', 'bottled beer', 'rice', 'whole milk', 'softener', 'skin care', 'chicken'], ['desse

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp) # df_tmp를 장바구니 형태로 변경 (fit과 transform을 동시에 진행)
print(te_ary)

[[False False False ...  True  True False]
 [False False False ...  True False False]
 [False False False ...  True False False]
 ...
 [False False False ... False False False]
 [False False False ... False  True False]
 [False False False ... False False False]]


In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_) # label 정보는 te.columns_에 저장되어 있으므로, 해당 값을 칼럼명으로 하는 데이터 프레임으로 변경.
df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


## Step4. Association Rule Mining

연관분석으로는 지지도, 신뢰도, 향상도의 세가지 측면에서 분석이 가능하다.

# 지지도 (Support)
우선 지지도를 확인하여 빈도가 많고 구성비가 높은 좋은 규칙을 찾고자 하였다.

- 연관분석 함수(`apriori, association_rules`)를 사용하여 분석 진행  
  - 지지도(support): 범용성 0.1
    - rule의 조건이 일어날 확률
    - 전체 중에서 상품 A가 몇 번이나 판매되었는가  
  $s(X→Y)=P(X \cap Y)$  

두 항목 X와 Y의 지지도는 전체 거래 건수 중에서 항목집합 X와 Y를 모두 포함하는 거래 건수의 비율을 말합니다.   지지도는 좋은 규칙(빈도가 많은, 구성비가 높은)을 찾거나, 불필요한 연산을 줄일 때(pruning, 가지치기)의 기준으로 사용합니다.

출처: https://rfriend.tistory.com/191 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

In [ ]:
''' Apriori Algorithm 활용
- 빈번하게 발생하는 아이템 셋에 대해서만 계산 (minimum support를 넘지 못하는 아이템을 포함하는 모든 아이템셋은 minimum support를 넘지 못함)
1. Support
- Minimum support를 지정하여, 등장 횟수가 일정 비율 이상의 아이템 set만 선정
  - 1) 각 item들에 대해 지지도를 계산 후 minimum 지지도가 넘는 items를 선정
  - 2) 1에서 구한 items들의 2개 조합으로 구성한 set을 생성 후 지지도 계산 (minimum 이상)
  - 3) 2에서 구한 set들의 조합으로, 3개의 item으로 구성된 항목에 대해 지지도 계산
  - 4) minimum 값을 넘는 조합이 없을 때까지 조합에 포함되는 item 개수를 늘려줌
'''
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True) # 여기서는 0.05를 기준으로 선정
frequent_itemsets.sort_values(by='support', ascending=False).head(10)

,support,itemsets
55,0.458184,(whole milk)
37,0.376603,(other vegetables)
41,0.349666,(rolls/buns)
47,0.313494,(soda)
56,0.282966,(yogurt)
51,0.233710,(tropical fruit)
42,0.230631,(root vegetables)
5,0.213699,(bottled water)
44,0.206003,(sausage)
116,0.191380,"(other vegetables, whole milk)"


* whole milk 를 구매한 경우가 많다.(support = 0.458)
* whole milk, other vegetables 를 같이 구매한 경우가 많다.(support = 0.19)
* whole milk, other vegetables, rolls/buns 순으로 노출시키는 것이 효과적으로 보인다. 

## 신뢰도 (Confidence)
다음으로는 신뢰도를 유용한 규칙을 찾고자 한다.

  - 신뢰도(confidence) 0.7
    - rule의 조건이 일어났을 때, 결과가 발생할 확률
    - A를 구입한 주문 중 B도 동시에 구입한 주문 수  
  $c(X → Y)=\frac{P(X \cap Y)}{P(X)}=P(B|A)$  

In [ ]:
'''
2. Confidence
- 앞서 선정한 items (많이 구매된 메뉴)을 사용하여 함께 구매된 items를 확인
- antecedents (조건절), consequents (결과절): Antecedents를 구매했을 때 consequents를 구매
'''
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False).head(30) # 최소 confidence를 0.1로 지정

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
195,"(other vegetables, bottled water)",(whole milk),0.093894,0.458184,0.056183,0.598361,1.305941,0.013162,1.349012
237,"(other vegetables, yogurt)",(whole milk),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507
249,"(rolls/buns, yogurt)",(whole milk),0.111339,0.458184,0.065931,0.592166,1.292420,0.014917,1.328521
206,"(rolls/buns, other vegetables)",(whole milk),0.146742,0.458184,0.082093,0.559441,1.220996,0.014859,1.229837
256,"(soda, yogurt)",(whole milk),0.097486,0.458184,0.054387,0.557895,1.217622,0.009720,1.225537
225,"(other vegetables, soda)",(whole milk),0.124166,0.458184,0.069266,0.557851,1.217528,0.012375,1.225416
231,"(other vegetables, tropical fruit)",(whole milk),0.091329,0.458184,0.050539,0.553371,1.207749,0.008693,1.213123
243,"(rolls/buns, soda)",(whole milk),0.119805,0.458184,0.065162,0.543897,1.187072,0.010269,1.187926
177,(shopping bags),(whole milk),0.168291,0.458184,0.091329,0.542683,1.184422,0.014220,1.184772
218,"(sausage, other vegetables)",(whole milk),0.092868,0.458184,0.050282,0.541436,1.181702,0.007732,1.181551


* other vegetables, bottled water를 산 사람들 중 whole milk 를 산 사람들이 많았다. (confidence = 0.59)
* bottled water, whole milk 를 산 사람들 중 other vegetables 를 산 사람들이 많았다. (confidence = 0.5)
* other vegetables 를 산 사람이 whole milk 를 살 확률(confidence =0.59) > whole milk를 산 사람이 other vetables 를 살 확률(confidence =0.5)

## 향상도 (Lift)

항목집합 X가 주어지지 않았을 때의 항목집합 Y의 확률 대비 항목집합 X가 주어졌을 대 항목집합 Y의 확률 증가 비율을 말합니다. 다른말로 표현하자면, 향상도가 1보다 크거나(+관계) 작다면(-관계) 우연적 기회(random chance)보다 우수함을 의미합니다. (X와 Y가 서로 독립이면 Lift = 1)



출처: https://rfriend.tistory.com/191 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

  - 향상도(lift) 
    - 조건과 결과가 각각 독립적이라고 가정할 경우보다 얼마나 더 좋은지를 평가
    - 두 상품의 관계를 직접적으로 나타낸 것  
   $Lift(X, Y)=\frac{P(A \cap B)}{P(A) \times P(B)}=\frac{P(B|A)}{P(B)}$  
    - X가 기본 아이템일 경우 항상 confidence가 1이 나오는 경우 두 아이템 간 관계를 파악할 수 없으므로 향상도를 비교

In [ ]:
'''
3. Lift
- 1보다 클 경우 A를 샀을 때 B을 살 확률이 높아지는 것을 의미
- 1보다 작을 경우 A를 샀을 때 B을 사지 않을 확률이 높은 것을 의미  
'''
association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
197,(bottled water),"(other vegetables, whole milk)",0.213699,0.191380,0.056183,0.262905,1.373732,0.015285,1.097036
192,"(other vegetables, whole milk)",(bottled water),0.191380,0.213699,0.056183,0.293566,1.373732,0.015285,1.113056
195,(other vegetables),"(whole milk, bottled water)",0.376603,0.112365,0.056183,0.149183,1.327657,0.013865,1.043273
194,"(whole milk, bottled water)",(other vegetables),0.112365,0.376603,0.056183,0.500000,1.327657,0.013865,1.246793
239,(yogurt),"(other vegetables, whole milk)",0.282966,0.191380,0.071832,0.253853,1.326434,0.017678,1.083727
...,...,...,...,...,...,...,...,...,...
110,(other vegetables),(soda),0.376603,0.313494,0.124166,0.329700,1.051695,0.006103,1.024178
59,(coffee),(whole milk),0.114931,0.458184,0.055156,0.479911,1.047420,0.002497,1.041776
58,(whole milk),(coffee),0.458184,0.114931,0.055156,0.120381,1.047420,0.002497,1.006196
113,(tropical fruit),(other vegetables),0.233710,0.376603,0.091329,0.390779,1.037642,0.003313,1.023269


- 일반적으로 association rule을 사용하여 분석할 때는 다음과 같은 과정을 거침  
    1) 특정 지지도 이상인 set을 찾음  
    2) 해당 set의 조합으로 만들 수 있는 rule 중 신뢰도 이하의 rule을 screening out   
    3) 향상도를 정렬하여 rule을 평가

In [ ]:
association_rules(frequent_itemsets,metric='lift',min_threshold=1).sort_values(by=['lift'],ascending=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
113,(tropical fruit),(other vegetables),0.233710,0.376603,0.091329,0.390779,1.037642,0.003313,1.023269
112,(other vegetables),(tropical fruit),0.376603,0.233710,0.091329,0.242507,1.037642,0.003313,1.011614
58,(whole milk),(coffee),0.458184,0.114931,0.055156,0.120381,1.047420,0.002497,1.006196
59,(coffee),(whole milk),0.114931,0.458184,0.055156,0.479911,1.047420,0.002497,1.041776
111,(soda),(other vegetables),0.313494,0.376603,0.124166,0.396072,1.051695,0.006103,1.032237
...,...,...,...,...,...,...,...,...,...
239,(yogurt),"(other vegetables, whole milk)",0.282966,0.191380,0.071832,0.253853,1.326434,0.017678,1.083727
194,"(whole milk, bottled water)",(other vegetables),0.112365,0.376603,0.056183,0.500000,1.327657,0.013865,1.246793
195,(other vegetables),"(whole milk, bottled water)",0.376603,0.112365,0.056183,0.149183,1.327657,0.013865,1.043273
192,"(other vegetables, whole milk)",(bottled water),0.191380,0.213699,0.056183,0.293566,1.373732,0.015285,1.113056


* bottled water를 산 사람들이	other vegetables, whole milk를 주문할 확률에는 양의 상관관계가 있음(lift = 1.37)
* lift 가 1 이하인 경우는 없었음 -> 음의 상관관계를 가진 변수는 없음

In [ ]:
# Finding the most frequent items sold together
frequent_itemsets = apriori(df, min_support=0.025, use_colnames=True, max_len=3).sort_values(by='support')
frequent_itemsets.head(25)

,support,itemsets
161,0.025141,"(butter, shopping bags)"
69,0.025141,(spread cheese)
405,0.025141,"(whole milk, sliced cheese)"
412,0.025141,"(specialty bar, whole milk)"
85,0.025141,"(beef, pip fruit)"
248,0.025141,"(shopping bags, domestic eggs)"
467,0.025141,"(citrus fruit, whole milk, tropical fruit)"
480,0.025141,"(whole milk, yogurt, frankfurter)"
119,0.025141,"(chocolate, bottled water)"
239,0.025141,"(root vegetables, dessert)"


가장 많이 같이 팔린 물건은 shopping bags, butter, sperad cheese 순서이다. 

- 지지도, 신뢰도 등의 minimum threshold를 결정할 때 주관성이 들어갈 수 있음
- 목적에 따라 중요하게 판단할 기준이 다를 수 있음
  - 신뢰도(confidence)가 동일하다고 했을 때, 소비자노출이 중요하다면 지지도(support)가 높은 Rule이, 노출 대비 소비자 반응률(구매여부)이 중요한 경우 향상도(lift)가 높은 Rule이 더 좋은 규칙

## 참고사이트
https://www.kaggle.com/code/atasaygin/groceries-market-basket-and-rfm-analysis

https://www.kaggle.com/code/priy998/groceries-analysis-using-apriori-algorithm